In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')

In [ ]:
import sklearn

In [ ]:
from sklearn import datasets
from sklearn import model_selection

def create_folds(data):
 # we create a new column called kfold and fill it with -1
 data["kfold"] = -1
 
 
 data = data.sample(frac=1).reset_index(drop=True)
 
 # initiate the kfold class from model_selection module
 kf = model_selection.StratifiedKFold(n_splits=5)
 
 # fill the new kfold column
 # note that, instead of targets, we use bins!
 for f, (t_, v_) in enumerate(kf.split(X=data, y=data.target)):
     data.loc[v_, 'kfold'] = f
 
 
 return data

In [ ]:
df=create_folds(df)


In [ ]:
df.to_csv('kfold_train.csv')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xg
import joblib
import lightgbm as lgb

In [ ]:
d=dict()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def run_folds_lgbm(df,fold):
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    x_train=df_train.drop(['target','id','kfold'],axis=1).values
    y_train=df_train.target.values
    
    x_test=df_test.drop(['target','id','kfold'],axis=1).values
    y_test=df_test.target.values
    #parameters of lgbm i got from optuna u can see my notebook
    
    param={"objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
           'lambda_l1': 1.4536206738653063e-07, 'lambda_l2': 6.458971864500743e-08, 'num_leaves': 131, 
           'feature_fraction': 0.5474311915512885,
           'bagging_fraction': 0.9388678532827928,
           'bagging_freq': 6, 'min_child_samples': 13}
    
    clf= lgb.LGBMClassifier(**param)#xg.XGBClassifier(**param)
    clf.fit(x_train,y_train)
    
    d['y_pred'+str(fold)+'lgbm'] = clf.predict_proba(x_test)[:,1]
    roc_auc_score = sklearn.metrics.roc_auc_score(y_test,d['y_pred'+str(fold)+'lgbm'])
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    
    File_name = 'model_lgbmclf' + str(fold)
    joblib.dump(
    clf,File_name)
    
    
for i in range(5):
    run_folds_lgbm(df,i) 

In [ ]:
model_0_lgbmclf= joblib.load('./model_lgbmclf0')
model_1_lgbmclf =joblib.load('./model_lgbmclf1')
model_2_lgbmclf= joblib.load('./model_lgbmclf2')
model_3_lgbmclf= joblib.load('./model_lgbmclf3')
model_4_lgbmclf= joblib.load('./model_lgbmclf4')

In [ ]:
d1=dict()

In [ ]:
def run_folds_xgb(df,fold):
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    x_train=df_train.drop(['target','id','kfold'],axis=1).values
    y_train=df_train.target.values
    
    x_test=df_test.drop(['target','id','kfold'],axis=1).values
    y_test=df_test.target.values
    #parameters of xgboost i got from optuna u can see my notebook
    param={'tree_method':'gpu_hist','lambda': 0.5033009770384954, 'alpha': 0.5585785710700676, 'colsample_bytree': 0.8, 
           'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 263,
          'use_label_encoder':False,'n_estimators': 4000}

    
    clf= xg.XGBClassifier(**param)
    clf.fit(x_train,y_train)
    
    d1['y_pred'+str(fold)+'xgb'] = clf.predict_proba(x_test)[:,1]
    roc_auc_score = sklearn.metrics.roc_auc_score(y_test,d1['y_pred'+str(fold)+'xgb'])
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    
    File_name = 'model_xgbclf' + str(fold)
    joblib.dump(
    clf,File_name)
    
    
for i in range(5):
    run_folds_xgb(df,i)    

In [ ]:
model_0_xgbclf= joblib.load('./model_xgbclf0')
model_1_xgbclf =joblib.load('./model_xgbclf1')
model_2_xgbclf= joblib.load('./model_xgbclf2')
model_3_xgbclf= joblib.load('./model_xgbclf3')
model_4_xgbclf= joblib.load('./model_xgbclf4')

In [ ]:
#this is ensembling u can read more about in Aproach Almost Any Machine Learning Book by Abhisek Thakur
import numpy as np
from functools import partial
from scipy.optimize import fmin
from sklearn import metrics

class OptimizeAUC:
    """
    Class for optimizing AUC.
    This class is all you need to find best weights for 
    any model and for any metric and for any types of predictions.
    With very small changes, this class can be used for optimization of 
    weights in ensemble models of _any_ type of predictions
    """
    def __init__(self):
         self.coef_ = 0

    def _auc(self, coef, X, y):
         """
         This functions calulates and returns AUC.
         :param coef: coef list, of the same length as number of models
         :param X: predictions, in this case a 2d array
         :param y: targets, in our case binary 1d array
         """
         # multiply coefficients with every column of the array
         # with predictions.
         # this means: element 1 of coef is multiplied by column 1
         # of the prediction array, element 2 of coef is multiplied 
         # by column 2 of the prediction array and so on!
         x_coef = X * coef
         # create predictions by taking row wise sum
         predictions = np.sum(x_coef, axis=1)

         # calculate auc score
         auc_score = metrics.roc_auc_score(y, predictions)
         # return negative auc
         return -1.0 * auc_score
        
    def fit(self, X, y):
     # remember partial from hyperparameter optimization chapter?
     loss_partial = partial(self._auc, X=X, y=y)

     # dirichlet distribution. you can use any distribution you want
     # to initialize the coefficients
     # we want the coefficients to sum to 1
     initial_coef = np.random.dirichlet(np.ones(X.shape[1]), size=1)
     # use scipy fmin to minimize the loss function, in our case auc
     self.coef_ = fmin(loss_partial, initial_coef, disp=True)
    def predict(self, X):
        # this is similar to _auc function
        x_coef = X * self.coef_

        predictions = np.sum(x_coef, axis=1)
        return predictions

In [ ]:
pred_lgbm = model_0_lgbmclf.predict_proba(df[df.kfold==1].iloc[:,1:-2])[:, 1]
pred_xgb  = model_0_xgbclf.predict_proba(df[df.kfold==1].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold0_preds = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))


pred_lgbm = model_1_lgbmclf.predict_proba(df[df.kfold==2].iloc[:,1:-2])[:, 1]
pred_xgb  = model_1_xgbclf.predict_proba(df[df.kfold==2].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold1_preds = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))


pred_lgbm = model_2_lgbmclf.predict_proba(df[df.kfold==3].iloc[:,1:-2])[:, 1]
pred_xgb  = model_2_xgbclf.predict_proba(df[df.kfold==3].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold2_preds = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))


pred_lgbm = model_3_lgbmclf.predict_proba(df[df.kfold==4].iloc[:,1:-2])[:, 1]
pred_xgb  = model_3_xgbclf.predict_proba(df[df.kfold==4].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold3_preds = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))



pred_lgbm = model_4_lgbmclf.predict_proba(df[df.kfold==0].iloc[:,1:-2])[:, 1]
pred_xgb  = model_4_xgbclf.predict_proba(df[df.kfold==0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold4_preds = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

In [ ]:
pred_lgbm = d['y_pred0lgbm']#model_0_lgbmclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
pred_xgb  = d1['y_pred0xgb']#model_0_xgbclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold0_test = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

pred_lgbm = d['y_pred1lgbm']#model_0_lgbmclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
pred_xgb  = d1['y_pred1xgb']#model_0_xgbclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold1_test = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

pred_lgbm = d['y_pred2lgbm']#model_0_lgbmclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
pred_xgb  = d1['y_pred2xgb']#model_0_xgbclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold2_test = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

pred_lgbm = d['y_pred3lgbm']#model_0_lgbmclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
pred_xgb  = d1['y_pred3xgb']#model_0_xgbclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold3_test = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

pred_lgbm = d['y_pred4lgbm']#model_0_lgbmclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
pred_xgb  = d1['y_pred4xgb']#model_0_xgbclf.predict_proba(df[df.kfold!=0].iloc[:,1:-2])[:, 1]
avg_pred = (pred_lgbm + pred_xgb) / 2

fold4_test = np.column_stack((
 pred_lgbm,
 pred_xgb,
#  pred_xgbc,
 avg_pred
))

In [ ]:
opt0 = OptimizeAUC()
# dont forget to remove the average column
opt0.fit(fold0_preds[:, :-1], df[df.kfold==1].target.values)

opt_preds_fold2 = opt0.predict(fold0_test[:, :-1])
auc = metrics.roc_auc_score(df[df.kfold==0].target.values, opt_preds_fold2)
print(f"Optimized AUC, Fold 0 = {auc}")
print(f"Coefficients = {opt0.coef_}")

In [ ]:
opt1 = OptimizeAUC()
# dont forget to remove the average column
opt1.fit(fold1_preds[:, :-1], df[df.kfold==2].target.values)
# for i in range(1,5):
opt_preds_fold2 = opt1.predict(fold1_test[:, :-1])
auc = metrics.roc_auc_score(df[df.kfold==1].target.values, opt_preds_fold2)
print(f"Optimized AUC, Fold 1 = {auc}")
print(f"Coefficients = {opt1.coef_}")

In [ ]:
opt2 = OptimizeAUC()
# dont forget to remove the average column
opt2.fit(fold2_preds[:, :-1], df[df.kfold==3].target.values)
# for i in range(1,5):
opt_preds_fold1 = opt2.predict(fold2_test[:, :-1])
auc = metrics.roc_auc_score(df[df.kfold==2].target.values, opt_preds_fold1)
print(f"Optimized AUC, Fold 2 = {auc}")
print(f"Coefficients = {opt2.coef_}")

In [ ]:
opt3 = OptimizeAUC()
# dont forget to remove the average column
opt3.fit(fold3_preds[:, :-1], df[df.kfold==4].target.values)
# for i in range(1,5):
opt_preds_fold1 = opt3.predict(fold3_test[:, :-1])
auc = metrics.roc_auc_score(df[df.kfold==3].target.values, opt_preds_fold1)
print(f"Optimized AUC, Fold 3 = {auc}")
print(f"Coefficients = {opt3.coef_}")

In [ ]:
opt4 = OptimizeAUC()
# dont forget to remove the average column
opt4.fit(fold4_preds[:, :-1], df[df.kfold==0].target.values)
# for i in range(1,5):
opt_preds_fold1 = opt4.predict(fold4_test[:, :-1])
auc = metrics.roc_auc_score(df[df.kfold==4].target.values, opt_preds_fold1)
print(f"Optimized AUC, Fold 2 = {auc}")
print(f"Coefficients = {opt4.coef_}")

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_test = df_test.drop(columns = 'id')

In [ ]:
y_final_32 = model_3_xgbclf.predict_proba(df_test)[:,1]
y_final_02 = model_0_xgbclf.predict_proba(df_test)[:,1]
y_final_12 = model_1_xgbclf.predict_proba(df_test)[:,1]
y_final_22 = model_2_xgbclf.predict_proba(df_test)[:,1]
y_final_42 = model_4_xgbclf.predict_proba(df_test)[:,1]

In [ ]:
y_final_31 = model_3_lgbmclf.predict_proba(df_test)[:,1]
y_final_01 = model_0_lgbmclf.predict_proba(df_test)[:,1]
y_final_11 = model_1_lgbmclf.predict_proba(df_test)[:,1]
y_final_21 = model_2_lgbmclf.predict_proba(df_test)[:,1]
y_final_41 = model_4_lgbmclf.predict_proba(df_test)[:,1]

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
submission['target'] = y_final_32
submission.to_csv('prediction_xgb_32.csv',index = False)

In [ ]:
submission['target'] = y_final_02
submission.to_csv('prediction_xgb_02.csv',index = False)


submission['target'] = y_final_12
submission.to_csv('prediction_xgb_12.csv',index = False)

submission['target'] = y_final_22
submission.to_csv('prediction_xgb_22.csv',index = False)

submission['target'] = y_final_42
submission.to_csv('prediction_xgb_42.csv',index = False)

submission['target'] = y_final_31
submission.to_csv('prediction_lgbm_31.csv',index = False)

submission['target'] = y_final_11
submission.to_csv('prediction_lgbm_11.csv',index = False)

submission['target'] = y_final_21
submission.to_csv('prediction_lgbm_21.csv',index = False)

submission['target'] = y_final_01
submission.to_csv('prediction_lgbm_01.csv',index = False)

submission['target'] = y_final_41
submission.to_csv('prediction_lgbm_41.csv',index = False)

In [ ]:
fold0_preds = np.column_stack((
 y_final_01,
 y_final_02,
))

fold1_preds = np.column_stack((
 y_final_11,
 y_final_12,
))

fold2_preds = np.column_stack((
 y_final_21,
 y_final_22,
))

fold3_preds = np.column_stack((
 y_final_31,
 y_final_32,
))

fold4_preds = np.column_stack((
 y_final_41,
 y_final_42,
))

In [ ]:
opt_preds_fold0 = opt0.predict(fold0_preds)

opt_preds_fold1 = opt1.predict(fold1_preds)

opt_preds_fold2 = opt2.predict(fold2_preds)

opt_preds_fold3 = opt3.predict(fold3_preds)

opt_preds_fold4 = opt4.predict(fold4_preds)

In [ ]:
y_final=(opt_preds_fold0 + opt_preds_fold1 + opt_preds_fold2 + opt_preds_fold3 + opt_preds_fold4)/5

In [ ]:
submission['target'] = y_final
submission.to_csv('prediction_final.csv',index = False)

In [ ]:
# [0.03623127 1.04846433]
# [0.05643082 1.3788837 
#  [0.02046658 0.90954134]
# [0.07897787 1.20085774]
#  [0.04216823 1.16449406]

In [ ]:
c1=0.03623127 +0.05643082+0.02046658+0.07897787+0.04216823  #this values i got by running the above 5 cells
c2=1.04846433+1.3788837+0.90954134+1.20085774+1.16449406   #this values i got by running the above 5 cells

c1=c1/5
c2=c2/5

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_test = df_test.drop(columns = 'id')

In [ ]:
y_final_3 = model_3_xgbclf.predict_proba(df_test)[:,1]
y_final_0 = model_0_xgbclf.predict_proba(df_test)[:,1]
y_final_1 = model_1_xgbclf.predict_proba(df_test)[:,1]
y_final_2 = model_2_xgbclf.predict_proba(df_test)[:,1]
y_final_4 = model_4_xgbclf.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg2 = (y_final_0 + y_final_1 +y_final_2 + y_final_3 + y_final_4)/5

In [ ]:
y_final_3 = model_3_lgbmclf.predict_proba(df_test)[:,1]
y_final_0 = model_0_lgbmclf.predict_proba(df_test)[:,1]
y_final_1 = model_1_lgbmclf.predict_proba(df_test)[:,1]
y_final_2 = model_2_lgbmclf.predict_proba(df_test)[:,1]
y_final_4 = model_4_lgbmclf.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg1 = (y_final_0 + y_final_1 +y_final_2 + y_final_3 + y_final_4)/5

In [ ]:
final_preds = np.column_stack((
 y_final_avg1,
 y_final_avg2
))



x_coef = final_preds * [c1,c2]
predictions = np.sum(x_coef, axis=1)
# return predictions

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
submission['target'] = predictions
submission.to_csv('prediction3.csv',index = False)

In [ ]:
submission['target'] = y_final_avg1
submission.to_csv('prediction_lgbm.csv',index = False)

In [ ]:
submission['target'] = y_final_avg2
submission.to_csv('prediction_xgb.csv',index = False)